In [1]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_community.vectorstores import Chroma
from langchain_community.document_loaders import PyPDFLoader
from langchain_core.prompts import ChatPromptTemplate

from langchain.retrievers import ParentDocumentRetriever
from langchain.storage import InMemoryStore


from langchain_core.runnables import RunnablePassthrough, RunnableParallel
from langchain_core.output_parsers import StrOutputParser


import os
from dotenv import dotenv_values


In [2]:
config = dotenv_values(dotenv_path='../.env')

In [3]:
os.environ['OPENAI_API_KEY'] = config['OPENAI_API_KEY']

# Definição de LLMs

In [4]:
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.4, max_tokens=500)

# Carregar PDF e instanciar

In [5]:
path_file = './os-sertoes.pdf'

loader = PyPDFLoader(path_file, extract_images=False)

pages = loader.load_and_split()

# Definição de splitter

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=100,
    length_function=len,
    add_start_index=True
)

In [7]:
chunks = text_splitter.split_documents(pages)

In [ ]:
store = Chroma(
    embedding_function=embeddings,
    persist_directory="naive_rag_db"
)

In [ ]:
vectorstore = store.from_documents(chunks, embedding=embeddings, persist_directory="parent_rag_db")

In [ ]:
retrievel = vectorstore.as_retriever(search_kwargs={"k": 5})

In [ ]:
template_quests = '''
Você é um especialista no livro Os sertões de Euclides da Cunha. Deve responder objetivamente a questão do usuario com base no conteudo fornecido de contexto

Questão do usuario: {query}


Conteudo: {content}
'''

rag_prompt = ChatPromptTemplate.from_template(template_quests)

In [ ]:
setup_retrievel = RunnableParallel({
    "query": RunnablePassthrough(),
    "content": retrievel,
})

In [ ]:
output_parser = StrOutputParser()

In [ ]:
chain_retriever = setup_retrievel | rag_prompt | llm | output_parser

In [ ]:
chain_retriever.invoke('Qual é a visão de Euclides da Cunha sobre o ambiente natural do sertão nordestino e como ele influencia a vida dos habitantes?')

'A visão de Euclides da Cunha sobre o ambiente natural do sertão nordestino é de um local marcado por condições extremas e adversas, onde os elementos físicos como o clima, a geologia e a topografia se influenciam mutuamente. Ele destaca a mesologia do local, ressaltando a dificuldade de definição de qual agente é preponderante nesse ambiente. Euclides da Cunha descreve a região como selvagem, com uma flora extravagante e um clima instável, marcado por intermitências de cheias e estiagens. Essas condições climáticas extremas influenciam a vida dos habitantes do sertão nordestino, levando a um estilo de vida difícil e adaptado às adversidades do ambiente.'